In [7]:
!git clone https://github.com/smhassanerfani/atex.git
!pip install timm  # install timm for the model

# git lfs pull

fatal: destination path 'atex' already exists and is not an empty directory.


In [8]:
!pip install torch torchvision scikit-learn

In [2]:
import os
import torch
import timm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [5]:
# Load the dataset
data_dir = "atex/data/atex"
full_dataset = ImageFolder(root=os.path.join(data_dir, "train"), transform=transform)
val_dataset = ImageFolder(root=os.path.join(data_dir, "val"), transform=transform)
test_dataset = ImageFolder(root=os.path.join(data_dir, "test"), transform=transform)
num_classes = len(full_dataset.classes)

train_loader = DataLoader(full_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [10]:
# Initialize the model
model = timm.create_model('mobilenetv3_small_100.lamb_in1k', pretrained=True, num_classes=num_classes)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# Training loop
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

device = next(model.parameters()).device
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {total_loss/len(train_loader):.4f}")

Epoch 1/10, Train Loss: 1.4098
Epoch 2/10, Train Loss: 0.7030
Epoch 3/10, Train Loss: 0.5248
Epoch 4/10, Train Loss: 0.3600
Epoch 5/10, Train Loss: 0.3002
Epoch 6/10, Train Loss: 0.2512
Epoch 7/10, Train Loss: 0.2182
Epoch 8/10, Train Loss: 0.1698
Epoch 9/10, Train Loss: 0.1846
Epoch 10/10, Train Loss: 0.1540


In [12]:
# Evaluate
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

report = classification_report(all_labels, all_preds, target_names=full_dataset.classes, digits=4)
print(report)

              precision    recall  f1-score   support

       delta     0.9403    0.9061    0.9228       330
     estuary     0.8730    0.8800    0.8765       125
       flood     0.8789    0.7951    0.8349       283
    glaciers     0.9234    0.9542    0.9385       240
  hot_spring     0.7991    0.8507    0.8241       201
        lake     0.6442    0.8758    0.7424       153
        pool     0.9394    0.7848    0.8552        79
      puddle     0.8376    0.5833    0.6877       168
      rapids     0.8502    0.8813    0.8655       219
       river     0.7547    0.5479    0.6349        73
         sea     0.7769    0.8704    0.8210       108
        snow     0.7937    0.8944    0.8411       142
       swamp     0.9365    0.9415    0.9390       188
   waterfall     0.7812    0.7812    0.7812        96
     wetland     0.8788    0.9355    0.9062        93

    accuracy                         0.8491      2498
   macro avg     0.8405    0.8321    0.8314      2498
weighted avg     0.8545   